In [1]:

import re
from collections import *
import sys
from nltk.stem.snowball import SnowballStemmer
import os
import time

In [2]:
tags = {1:'title',2:'text', 3:'category', 4 : 'infobox'}
dic = defaultdict(float)


In [3]:
offsets = [open('offsets/title.txt','r',encoding='ISO-8859-1'),open('offsets/text.txt','r',encoding='ISO-8859-1'),open('offsets/category.txt','r',encoding='ISO-8859-1'),open('offsets/infobox.txt','r',encoding='ISO-8859-1')]

In [4]:
scores = [open('score/title.txt','r',encoding='ISO-8859-1'),open('score/text.txt','r',encoding='ISO-8859-1'),open('score/category.txt','r',encoding='ISO-8859-1'),open('score/infobox.txt','r',encoding='ISO-8859-1')]

In [5]:
title_file  = open('doc_title.txt','r',encoding='ISO-8859-1')
doc_offset = []
lines = title_file.readlines()
cur =0
for line in lines:
    doc_offset.append(cur)
    cur+=len(line)+1

In [6]:
word_pos  = [defaultdict(list) for i in range(4)]

for i in range(4):
    lines = offsets[i].readlines()
    for line in lines:
        temp = line.split(':')
        word = "".join(temp[0:-1])
        
        if len(temp[-1])==0:
            continue
        page = int(temp[-1].strip())
        word_pos[i][word] = page
 #   offsets[i].close()

In [7]:
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, ' ', text)

In [8]:
stemmer = SnowballStemmer(language = 'english')

In [9]:
def tagq(words,typ):
    
    if typ==4:
        for i in range(4):
            tagq(words,i)
        return
    for word in words :
        word = stemmer.stem(word.lower())
        #word = remove_special_characters(word)
        if word in word_pos[typ]:
           # word = stemmer.stem(word.lower())
            #word = remove_special_characters(word)
            page = word_pos[typ][word]
            scores[typ].seek(page)
            line = scores[typ].readline()
            line = line.split(' ')
            for w in line[1:]:
                if len(w)<=1:
                    continue
                tmp = w.split(':')
                pg = tmp[0]
                tfidf =float(tmp[1])
                if pg not in dic:
                    dic[pg]=0
                dic[pg]+=tfidf

In [10]:
def process():
    results = sorted(dic.items(),key=lambda item: item[1],reverse=True)
    results = results[:min(10,len(results))]
    print(results)
    for cur in results:
        cur = cur[0]
        pos = doc_offset[int(cur)]
        title_file.seek(pos)
        title = title_file.readline().strip()
        title = title.replace(' ','_')
        print ("https://en.wikipedia.org/wiki/" + title)

In [11]:
def query():
    print('Any Queries?  (y or n) \n')
    q = input()
    if q=='y':
        clock_start = time.time()
        print('Select Query Type:')
        print('1 -Search By Title')
        print('2 -Search By Text')
        print('3 -Search By Category')
        print('4 -Search By Infobox')
        print('5 -Widesearch')
        typ = int(input())
        print('Type Your Query:\n')
        que = input().strip('\n').lower()
        words = que.split(' ')
        dic.clear()
        tagq(words,typ-1)
        process()
        print ("Query time = " +str(time.time()-clock_start))
        query()
        
    else :
        return
        
        
    
    
    

In [13]:
query()

Any Queries?  (y or n) 

y
Select Query Type:
1 -Search By Title
2 -Search By Text
3 -Search By Category
4 -Search By Infobox
5 -Widesearch
1
Type Your Query:

algorithm
[('187', 3.29), ('204', 3.29), ('387', 3.29), ('1199', 3.29), ('2461', 3.29), ('6412', 3.29), ('6477', 3.29), ('6494', 3.29), ('8185', 3.29), ('9604', 3.29)]
https://en.wikipedia.org/wiki/Algorithms_(journal)
https://en.wikipedia.org/wiki/Algorithm
https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance
https://en.wikipedia.org/wiki/Analysis_of_algorithms
https://en.wikipedia.org/wiki/Binary_search_algorithm
https://en.wikipedia.org/wiki/Euclids_algorithm
https://en.wikipedia.org/wiki/Euclidean_algorithm
https://en.wikipedia.org/wiki/Euclidean_Algorithm
https://en.wikipedia.org/wiki/GaussâLegendre_algorithm
https://en.wikipedia.org/wiki/International_Data_Encryption_Algorithm
Query time = 3.9620089530944824
Any Queries?  (y or n) 

m
